# Build / Use Vector Store
Below is a simple example of first building an in memory vector store from a Notion markdown text directory, and then using the store to query for search results.

## Env Setup
Before we get started, install the gems we need here:
`gem install boxcars dotenv`
and then create / edit .env to have OPENAI_ACCESS_TOKEN

In [ ]:
require 'dotenv/load'
require 'boxcars'

## Build the Vector Store

In [ ]:
hnswlib_vector = Boxcars::VectorStore::Hnswlib::BuildFromFiles.call(
  training_data_path: './notebooks/embeddings/Notion_DB/**/*.md',
  index_file_path: './notebooks/embeddings/hnswlib_notion_db_index.bin',
  json_doc_file_path: './notebooks/embeddings/hnswlib_notion_db_index.json',
  force_rebuild: false
)
puts :built

## Query the Store

In [ ]:
openai_client = Boxcars::Openai.open_ai_client

vector_search = Boxcars::VectorSearch.new(
  vector_documents: hnswlib_vector,
  openai_connection: openai_client
)

result = vector_search.call(query: 'What is the work from home policy?')

In [ ]:
puts result.first[:document].content

## More
You could of course use text files and get similar results. Other libraries can be brought it to handle PDFs and other binary formats. Add Issues and/or PRs for other types that you want supported.